## Jorge P. Rodríguez, 2023

In [1]:
import pandas as pd
import geopandas as gp
import pylab as plt
import numpy as np
plt.style.use('classic')
import matplotlib.colors as colors
from shapely import wkt
import math
import matplotlib.colorbar as chi
from shapely.geometry import Polygon
from shapely.geometry import LineString as line


In [2]:
df = pd.read_csv('./data/all_morethan664N_01deg.csv')
df.head()

,lat,lon,hours_total,geometry,hours_cargo,hours_fishing,hours_tanker,hours_passenger,cell
0,66.4,-170.5,9.0,"POLYGON ((-170.5 66.4, -170.4 66.4, -170.4 66....",3.0,0.0,0.0,0.0,5630495
1,66.4,-170.4,11.0,"POLYGON ((-170.4 66.4, -170.3 66.4, -170.3 66....",4.0,0.0,0.0,1.0,5630496
2,66.4,-170.3,22.0,"POLYGON ((-170.3 66.4, -170.20000000000002 66....",8.0,1.0,2.0,0.0,5630497
3,66.4,-170.2,29.0,"POLYGON ((-170.2 66.4, -170.1 66.4, -170.1 66....",11.0,0.0,3.0,0.0,5630498
4,66.4,-170.1,17.0,"POLYGON ((-170.1 66.4, -170 66.4, -170 66.5, -...",7.0,0.0,4.0,0.0,5630499


In [3]:
df = df[df['lat']>= 66.6]

In [4]:
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gp.GeoDataFrame(df, crs='epsg:4326')

In [5]:
gdf2 = gdf.to_crs('esri:102017')

In [6]:
world = gp.read_file('../continent/continent.shp')#A shapefile with the continent's shape
world = world.dissolve(aggfunc='sum')

In [7]:
#now we should normalize by cell area
R = 6371.
areas = []
pi = np.pi
for lat in df.lat.values:
    areas.append(R*R*abs(math.sin(pi*(lat+0.1)/180.)-math.sin(pi*(lat)/180.))*(0.1*pi/180.))
gdf2['area']=areas
gdf2['rho_total']=gdf2['hours_total']/gdf2['area']
gdf2['rho_cargo']=gdf2['hours_cargo']/gdf2['area']
gdf2['rho_passenger']=gdf2['hours_passenger']/gdf2['area']
gdf2['rho_fishing']=gdf2['hours_fishing']/gdf2['area']
gdf2['rho_tanker']=gdf2['hours_tanker']/gdf2['area']


                 

In [8]:
gdf2.head()

,lat,lon,hours_total,geometry,hours_cargo,hours_fishing,hours_tanker,hours_passenger,cell,area,rho_total,rho_cargo,rho_passenger,rho_fishing,rho_tanker
1608,66.6,-171.2,2.0,"POLYGON ((-396862.769 2563577.528, -401336.449...",0.0,0.0,0.0,0.0,5637688,49.005553,0.040812,0.000000,0.000000,0.000000,0.000000
1609,66.6,-171.1,2.0,"POLYGON ((-401336.449 2562880.968, -405808.907...",0.0,0.0,0.0,1.0,5637689,49.005553,0.040812,0.000000,0.020406,0.000000,0.000000
1610,66.6,-171.0,5.0,"POLYGON ((-405808.907 2562176.600, -410280.128...",2.0,0.0,0.0,0.0,5637690,49.005553,0.102029,0.040812,0.000000,0.000000,0.000000
1611,66.6,-170.9,18.0,"POLYGON ((-410280.128 2561464.428, -414750.100...",5.0,0.0,0.0,1.0,5637691,49.005553,0.367305,0.102029,0.020406,0.000000,0.000000
1612,66.6,-170.8,38.0,"POLYGON ((-414750.100 2560744.453, -419218.808...",12.0,1.0,7.0,1.0,5637692,49.005553,0.775422,0.244870,0.020406,0.020406,0.142841


In [9]:
cmap = plt.get_cmap('plasma')

In [10]:
cnorm = colors.LogNorm(vmin=gdf2['rho_total'].min(),vmax=gdf2['rho_total'].max()/100)

In [11]:
#let's add some references in the map
gaux = gp.GeoDataFrame(geometry=[line([(i,80) for i in range(-180,181)]),
                                 line([(i,70) for i in range(-180,181)]),
                                 line([(i,60) for i in range(-180,181)]),
                                line([(0,i) for i in range(80,60,-1)]),
                                line([(90,i) for i in range(80,60,-1)]),
                                     line([(-90,i) for i in range(80,60,-1)]),
                                 line([(180,i) for i in range(80,60,-1)])],
                       crs = 'epsg:4326')

In [12]:
gaux.head()

,geometry
0,"LINESTRING (-180.00000 80.00000, -179.00000 80..."
1,"LINESTRING (-180.00000 70.00000, -179.00000 70..."
2,"LINESTRING (-180.00000 60.00000, -179.00000 60..."
3,"LINESTRING (0.00000 80.00000, 0.00000 79.00000..."
4,"LINESTRING (90.00000 80.00000, 90.00000 79.000..."


In [13]:
gaux2 = gaux.to_crs('esri:102017')

In [14]:
pcaux = gp.GeoDataFrame(geometry=[line([(i,66.6) for i in range(-180,181)])],
                       crs = 'epsg:4326')

In [15]:
pcaux2 = pcaux.to_crs('esri:102017')

In [16]:
world2 = world.to_crs('esri:102017')

In [17]:
xmin = -2.7e6
xmax = 2.7e6
ymin = -2.7e6
ymax = 2.7e6
mask = Polygon([(xmin,ymin),(xmax,ymin),(xmax,ymax),(xmin,ymax),(xmin,ymin)])
world3 = gp.clip(world2,mask)

In [22]:
fig = plt.figure(figsize=(9,9))
fig.set_facecolor('w')
ax = plt.gca()
ax.set_position([0.06,0.08,0.86,0.86])
ax.set_facecolor('#eeeeee')
world3.plot(ax=ax,color='none',ec='#222222',zorder=2.1,lw=0.3)
world3.plot(ax=ax,color='#cccccc',ec='none',zorder=2.0)

gdf2.plot(column='rho_total',ax=ax,norm=cnorm,cmap='plasma',zorder=2.05,lw=1e-6,ec='None',alpha=1)
gaux2.plot(ax=ax,color='k',ls='dotted',zorder=2.2,lw=2)
pcaux2.plot(ax=ax,color='#f18d00',ls='-',zorder=2.2)
plt.text(-2.3e6,2.4e6,'Arctic Circle',color='#f18d00',size=20)

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.xticks([])
plt.yticks([])
axc = fig.add_axes([0.93,0.1,0.04,0.8])
clb=chi.ColorbarBase(axc,cmap=cmap,norm=cnorm,orientation='vertical',extend='max')

clb.set_label(r'Shipping density (h km$^{-2}$)', fontsize=18,color='#222222')
plt.yticks(color='#222222')
clb.ax.tick_params(labelsize=16)

ax1 = fig.add_axes([1.1,0.07,0.42,0.42])
ax1.set_facecolor('#eeeeee')
world3.plot(ax=ax1,color='none',ec='#222222',zorder=2.1,lw=0.3/2)
world3.plot(ax=ax1,color='#cccccc',ec='none',zorder=2.0)

gaux2.plot(ax=ax1,color='k',ls='dotted',zorder=2.2,lw=1)
pcaux2.plot(ax=ax1,color='#f18d00',ls='-',zorder=2.2)

gdf2.plot(column='rho_cargo',ax=ax1,norm=cnorm,cmap='plasma',zorder=2.05,lw=1e-6,ec='None',alpha=1.0)

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.xticks([])
plt.yticks([])
ax1.set_title('Cargo',fontsize=16,color='#222222')

ax2 = fig.add_axes([1.54,0.07,0.42,0.42])
ax2.set_facecolor('#eeeeee')
world3.plot(ax=ax2,color='none',ec='#222222',zorder=2.1,lw=0.3/2)
world3.plot(ax=ax2,color='#cccccc',ec='none',zorder=2.0)

gaux2.plot(ax=ax2,color='k',ls='dotted',zorder=2.2,lw=1)
pcaux2.plot(ax=ax2,color='#f18d00',ls='-',zorder=2.2)

gdf2.plot(column='rho_fishing',ax=ax2,norm=cnorm,cmap='plasma',zorder=2.05,lw=1e-6,ec='None',alpha=1.0)

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.xticks([])
plt.yticks([])
ax2.set_title('Fishing',fontsize=16,color='#222222')

ax3 = fig.add_axes([1.1,0.53,0.42,0.42])
ax3.set_facecolor('#eeeeee')
world3.plot(ax=ax3,color='none',ec='#222222',zorder=2.1,lw=0.3/2)
world3.plot(ax=ax3,color='#cccccc',ec='none',zorder=2.0)

gdf2.plot(column='rho_passenger',ax=ax3,norm=cnorm,cmap='plasma',zorder=2.05,lw=1e-6,ec='None',alpha=1.0)

gaux2.plot(ax=ax3,color='k',ls='dotted',zorder=2.2,lw=1)
pcaux2.plot(ax=ax3,color='#f18d00',ls='-',zorder=2.2)

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.xticks([])
plt.yticks([])
ax3.set_title('Passenger',fontsize=16,color='#222222')

ax4 = fig.add_axes([1.54,0.53,0.42,0.42])
ax4.set_facecolor('#eeeeee')
world3.plot(ax=ax4,color='none',ec='#222222',zorder=2.1,lw=0.3/2)
world3.plot(ax=ax4,color='#cccccc',ec='none',zorder=2.0)

gdf2.plot(column='rho_tanker',ax=ax4,norm=cnorm,cmap='plasma',zorder=2.05,lw=1e-6,ec='None',alpha=1.0)

gaux2.plot(ax=ax4,color='k',ls='dotted',zorder=2.2,lw=1)
pcaux2.plot(ax=ax4,color='#f18d00',ls='-',zorder=2.2)

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.xticks([])
plt.yticks([])
ax4.set_title('Tanker',fontsize=16,color='#222222');


In [23]:
fig.savefig('./fig1.png',bbox_inches='tight',dpi=300)

In [24]:
fig.savefig('./fig1.pdf',bbox_inches='tight')